# Сбор информации по новостройкам в Санкт-Петербурге

В этом ноутбуке содержится код сбора информации о новостройках в формате: 
* id жилого комплекса на сайте Циан
* название жилого комплекса на сайте Циан
* id застройщика на сайте Циан

Результат сохраняется в файле `complexes_data.csv` и используется в дальнейшем в ноутбуке `coordinates_grabber` для расширения информации о комплексах и в ноутбуке `links_generator` для того, чтобы сгенерировать ссылки на страницы с предложениями по этим ЖК. 

In [1]:
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
import time
import json

In [2]:
# Базовая ссылка, ведущая в раздел "Новостройки Санкт-Петербурга" на сайте Циан
base_link_str = "https://spb.cian.ru/newobjects/list?deal_type=sale&engine_version=2&offer_type=newobject&region=-2&p="

In [21]:
# Функция, позволяющая получить id и название жилого комплекса для всех представленных на странице ЖК
# Информация достается из исходного кода страницы
def get_complexes_info():
    
    all_complexes_ids = []
    all_complexes_names = []
    all_builders_ids = []
    
    # Итерация по всех страницам с информацией о новостройках
    for page_num in range(1, 22):
        link = base_link_str + str(page_num)
        
        # Получение текста исходного кода страницы
        search_page = requests.get(link.format(0))
        search_page = search_page.text
        search_page = BeautifulSoup(search_page, 'lxml')

        # Выделение части кода, содержащей список ЖК, представленных на странице
        complexes_part = re.findall(r"<script>!function\(\){(.*)}\(\)<\/script>", str(search_page))
        
        # Получение списка всех id жк, названий жк и id застройщика с помощью регулярных выражений
        ids = re.findall(r"\"id\":(.*?),", str(complexes_part))
        names = re.findall(r"\"name\":\"(.*?)\",", str(complexes_part))
        builders = re.findall(r"\"brand\":(.*?),", str(complexes_part))
        
        # Добавление информации, полученной с текущей страницы, в общие списки
        all_complexes_ids.extend(ids)
        all_complexes_names.extend(names)
        all_builders_ids.extend(builders)
        
        # Status message
        if page_num % 5 == 0:
            print("Information from page ", page_num, "just loaded")
        
    return all_complexes_ids, all_complexes_names, all_builders_ids

In [22]:
ids_arr, names_arr, builders_arr = get_complexes_info()

Information from page  5 just loaded
Information from page  10 just loaded
Information from page  15 just loaded
Information from page  20 just loaded


In [24]:
# Создаем DataFrame на основе полученных данных
df_dict = {'complex_id': ids_arr, 'complex_name': names_arr, 'builder_id': builders_arr}
df = pd.DataFrame.from_dict(df_dict)
df

,complex_id,complex_name,builder_id
0,6953,Граффити,2747
1,51431,"Институтский, 16",8075
2,53790,UP-квартал Пушкинский,9085
3,33807,Legenda Героев,8075
4,7202,Четыре горизонта,8077
...,...,...,...
520,6882,Пифагор,9121
521,6954,Ямбург,8084
522,14847,Мендельсон,1607
523,6926,"7 столиц, квартал Лондон",8079


In [25]:
# Сохраняем результаты для дальнейшего использования
df.to_csv('complexes_data.csv', encoding='utf-8')